In [ ]:
import os
import platform
import re
import time
from functools import reduce
from datetime import datetime
from typing import List, Tuple, Generator
from pyspark.sql.dataframe import DataFrame
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
import pyspark.sql.types as T


In [2]:
spark = (SparkSession.builder
         .master("local[*]")
         .appName("kafka-streaming")
         .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1,org.apache.spark:spark-streaming-kafka-0-10-assembly_2.12:3.0.1")
         .getOrCreate()
        )
spark

In [3]:

data = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "massive-events")
    .option("startingOffsets", "earliest")
    .load()
)

In [4]:
query = (
    data
    .selectExpr("cast(value as string)")
)
# {id: "cc4c98a9-7183-44ed-aeba-cadf4f758c02", desc: "test"}


In [5]:
schema = T.StructType().add("id", T.StringType()).add("desc", T.StringType())

In [6]:
events = (
    query
    .select(F.from_json(F.col("value"), schema).alias("data"))
    .select("data.*")
)
events.printSchema()

root
 |-- id: string (nullable = true)
 |-- desc: string (nullable = true)



In [7]:
events_query = (
    events
    .writeStream.queryName("dat")
    .format("memory")
    .outputMode("append")
    .start()
#     .awaitTermination()
)

In [21]:
spark.sql("select desc, count(*) from dat group by desc").show(200, truncate=False)

+-----------------------------+--------+
|desc                         |count(1)|
+-----------------------------+--------+
|test message: 140593120400960|35793   |
|test message: 140593095222848|35804   |
|test message: 140593112008256|35825   |
+-----------------------------+--------+



In [9]:
# x = (1, True)

In [10]:
# x[1]

In [11]:
# spark.stop()